__Title:__ Lab 2: Classificaiton  
__Authors:__ Butler, Derner, Holmes  
__Date:__ 2/5/23 

## Ruberic

| Category                  | Available | Requirements                                                                                                                                                                                                                                                                                                                                                                                                                      |
|---------------------------|-----------|-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| Total Points              | 100       |                                                                                                                                                                                                                                                                                                                                                                                                                                   |
| Data Preparation Part 1   | 10        | Define and prepare your class variables. Use proper variable representations (int, float, one-hot, etc.). Use pre-processing methods (as needed) for dimensionality reduction, scaling, etc. Remove variables that are not needed/useful for the analysis.                                                                                                                                                                        |
| Data Preparation Part 2   | 5         | Describe the final dataset that is used for classification/regression (include a description of any newly formed variables you created).                                                                                                                                                                                                                                                                                          |
| Modeling and Evaluation 1 | 10        | Choose and explain your evaluation metrics that you will use (i.e., accuracy, precision, recall, F-measure, or any metric we have discussed). Why are the measure(s) appropriate for analyzing the results of your modeling? Give a detailed explanation backing up any assertions.                                                                                                                                               |
| Modeling and Evaluation 2 | 10        | Choose the method you will use for dividing your data into training and testing splits (i.e., are you using Stratified 10-fold cross validation? Why?). Explain why your chosen method is appropriate or use more than one method as appropriate. For example, if you are using time series data then you should be using continuous training and testing sets across time.                                                       |
| Modeling and Evaluation 3 | 20        | Create three different classification/regression models for each task (e.g., random forest, KNN, and SVM for task one and the same or different algorithms for task two). Two modeling techniques must be new (but the third could be SVM or logistic regression). Adjust parameters as appropriate to increase generalization performance using your chosen metric. You must investigate different parameters of the algorithms! |
| Modeling and Evaluation 4 | 10        | Analyze the results using your chosen method of evaluation. Use visualizations of the results to bolster the analysis. Explain any visuals and analyze why they are interesting to someone that might use this model.                                                                                                                                                                                                             |
| Modeling and Evaluation 5 | 10        | Discuss the advantages of each model for each classification task, if any. If there are not advantages, explain why. Is any model better than another? Is the difference significant with 95% confidence? Use proper statistical comparison methods. You must use statistical comparison techniques—be sure they are appropriate for your chosen method of validation as discussed in unit 7 of the course.                       |
| Modeling and Evaluation 6 | 10        | Which attributes from your analysis are most important? Use proper methods discussed in class to evaluate the importance of different attributes. Discuss the results and hypothesize about why certain attributes are more important than others for a given classification task.                                                                                                                                                |
| Deployment                | 5         | How useful is your model for interested parties (i.e., the companies or organizations that might want to use it for prediction)? How would you measure the model's value if it was used by these parties? How would your deploy your model for interested parties? What other data should be collected? How often would the model need to be updated, etc.?                                                                       |
| Exceptional Work          | 10        | You have free reign to provide additional analyses. One idea: grid search parameters in a parallelized fashion and visualize the performances across attributes. Which parameters are most significant for making a good model for each classification algorithm?                                                                                                                                                                 |

__Libraries & Set-up__

In [8]:
# Import libraries
## Support Libraries
import pandas as pd
import numpy as np
import warnings

## Plotting
import plotly.express as px
import plotly.graph_objects as go
import matplotlib.pyplot as plt

## Preprocessing
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

## Model Selection
from sklearn.model_selection import train_test_split, GridSearchCV

## Models
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

## Feature Selection
from sklearn.feature_selection import SelectFromModel, VarianceThreshold, SelectPercentile

## Model Performance
from sklearn import metrics
from sklearn.metrics import confusion_matrix, classification_report, precision_score
from sklearn.metrics import roc_curve, auc

# Notebook Settings
warnings.filterwarnings(action='once')
pd.set_option('display.max_columns', None)


In [9]:
# Dataset
url = 'https://github.com/cdholmes11/MSDS-7331-ML1-Labs/blob/main/Mini-Lab_LogisticRegression_SVMs/Hotel%20Reservations.csv?raw=true'
hotel_df = pd.read_csv(url, encoding = "utf-8")

### Data Preparation Part 1
Define and prepare your class variables. Use proper variable representations (int, float, one-hot, etc.). Use pre-processing methods (as needed) for dimensionality reduction, scaling, etc. Remove variables that are not needed/useful for the analysis.
_____________

The below dataset updates are the direct result from our dataset analysis done in the previous lab.
 - [avg_price_per_room] - filtered to under $400 due to lack of observations over this price point and concerns of outlier influence
 - [no_of_previous_bookings_not_canceled] - dropped due to concerns of outlier influence and because it explains the same observations as [repated_guest]
 - [Booking_ID] - dropped because it's irrelevant for future predictions
 - [arrival_year] - dropped because this is not a time series model and thus, it's inclusion would limit our model's potential for classification of arrival years not found in our testing dataset.


In [10]:
# Dropping index column arrival_year
hotel_df_trim = hotel_df.drop(['Booking_ID', 'arrival_year', 'no_of_previous_bookings_not_canceled'], axis=1)
hotel_df_final = hotel_df_trim.loc[hotel_df_trim['avg_price_per_room'] < 400]

# Create data type groups
cat_features = ['type_of_meal_plan', 'required_car_parking_space', 'room_type_reserved', 'market_segment_type',
    'repeated_guest']
int_features = ['no_of_adults', 'no_of_children', 'no_of_weekend_nights', 'no_of_week_nights', 'arrival_month',
    'arrival_date', 'no_of_previous_cancellations', 'no_of_special_requests']
float_features = ['lead_time', 'avg_price_per_room']
cont_features = int_features + float_features

# Enforce data types
hotel_df_trim[cat_features] = hotel_df_trim[cat_features].astype('category')
hotel_df_trim[int_features] = hotel_df_trim[int_features].astype(np.int64)
hotel_df_trim[float_features] = hotel_df_trim[float_features].astype(np.float64)

# Making indexable list suitable for pipeline
cat_features_final = hotel_df_final[cat_features].columns
cont_features_final = hotel_df_final[cont_features].columns

In [64]:
# Train Test Split
X = hotel_df_final.drop('booking_status', axis = 1)
Y = hotel_df_final['booking_status']

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=110)

In [12]:
# Pipeline without features selection
numeric_features = cont_features_final
numeric_transformer = Pipeline(
    steps=[("scaler", StandardScaler())]
)

categorical_features = cat_features_final
categorical_transformer = Pipeline(
    steps=[
        ("encoder", OneHotEncoder(handle_unknown="ignore"))
    ]
)
preprocessor = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer, numeric_features),
        ("cat", categorical_transformer, categorical_features),
    ]
)

## Example - Remove for Final

In [ ]:
# Parameter grid for GridSearchCV. Will need new one for each model
param_grid = {
    'classifier__penalty' : ['l1', 'l2', 'elasticnet' ,'none'],
    'classifier__C' : [0.01, 0.1, 1, 10, 100, 1000],
    'classifier__solver' : ['lbfgs', 'liblinear', 'newton-cg', 'newton-cholesky', 'sag', 'saga'],
    }

In [ ]:
# Logistifc Regression Pipeline
clf = Pipeline(
    steps=[("preprocessor", preprocessor), ("classifier", LogisticRegression(max_iter=1000))]
)

grid_clf = GridSearchCV(
    clf,
    param_grid,
    verbose=False,
    n_jobs=-1,
    refit=True,
    cv=3
)
grid_clf.fit(X_train, y_train)

In [ ]:
print('Best Parameters:')
print(grid_clf.best_params_)

print('Internal CV score:')
print(grid_clf.best_score_)
y_pred_grid = grid_clf.predict(X_test)
print(classification_report(y_test, y_pred_grid))

### Data Preparation Part 2
Describe the final dataset that is used for classification/regression (include a description of any newly formed variables you created).
______

__MORE DETAIL IS NEEDED ON WHY THIS IS BEST FOR OUR DATASET__

### Modeling and Evaluation 1
Choose and explain your evaluation metrics that you will use (i.e., accuracy, precision, recall, F-measure, or any metric we have discussed). Why are the measure(s) appropriate for analyzing the results of your modeling? Give a detailed explanation backing up any assertions.
__________
We will be using AUC-ROC to as the primary metric for evaluating our models. This will provide us the optimal true positive and false postive rates for our models. The primary concern of the dataset is understanding the factors that lead to cancellation.

__MORE DETAIL IS NEEDED ON WHY THIS IS BEST FOR OUR DATASET__

### Modeling and Evaluation 2
Choose the method you will use for dividing your data into training and testing splits (i.e., are you using Stratified 10-fold cross validation? Why?). Explain why your chosen method is appropriate or use more than one method as appropriate. For example, if you are using time series data then you should be using continuous training and testing sets across time.
______________________
We have chosen to use an 80/20 train/test split with 10-fold cross validation. By keeping the 20% hold out and doing 10-fold cross-validation, we further reduce the chances of randomly splitting the data in a facorable way. We effectively get the benefits of 10-fold cross validations and offset the risk by keeping a 20% hold out for final model accuracy testing only. This removes our ability to train on the test set by repeatedly updating our model for better accuracy scores. 

### Modeling and Evaluation 3
Create three different classification/regression models for each task (e.g., random forest, KNN, and SVM for task one and the same or different algorithms for task two). Two modeling techniques must be new (but the third could be SVM or logistic regression). Adjust parameters as appropriate to increase generalization performance using your chosen metric. You must investigate different parameters of the algorithms!

__KNN__

__Decision Trees__

__Random Forest__

In [16]:
# Random Forest Paramters
rf_param_grid = {
    'classifier__n_estimators': [10, 50, 100, 200],
    'classifier__criterion': ['gini', 'entropy', 'log_loss'],
    'classifier__max_depth': [5, 10, None],
    'classifier__min_samples_split': [2, 5, 10, None],
    'classifier__max_features': ['sqrt', 'log2', None]
    }

In [42]:
# Model
rf = Pipeline(
    steps=[("preprocessor", preprocessor), ("classifier", RandomForestClassifier(random_state=110))]
)

grid_rf = GridSearchCV(
    rf,
    rf_param_grid,
    verbose=False,
    n_jobs=-1,
    refit=True,
    cv=3
)
grid_rf.fit(X_train, y_train)

C:\Users\corey\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\model_selection\_validation.py:378: FitFailedWarning:


324 fits failed out of a total of 1296.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
324 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\corey\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\model_selection\_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\corey\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-pack

In [67]:
# Performance Metrics
print('Best Parameters:')
print(grid_rf.best_params_)

print('Internal CV score:')
print(f'{grid_rf.best_score_:.4f}')
y_pred_rf = grid_rf.predict(X_test)

# Converting to Int
y_pred_rf = np.where(y_pred_rf == 'Canceled', 1, 0)
y_test_num = np.where(y_test == 'Canceled', 1, 0)
print(classification_report(y_test_num, y_pred_rf))

Best Parameters:
{'classifier__criterion': 'gini', 'classifier__max_depth': None, 'classifier__max_features': 'sqrt', 'classifier__min_samples_split': 2, 'classifier__n_estimators': 200}
Internal CV score:
0.8983
              precision    recall  f1-score   support

           0       0.91      0.95      0.93      4824
           1       0.88      0.81      0.84      2431

    accuracy                           0.90      7255
   macro avg       0.89      0.88      0.88      7255
weighted avg       0.90      0.90      0.90      7255



In [82]:
# Feature Importance
f_imp = grid_rf.best_estimator_.named_steps['classifier'].feature_importances_
feature_importance = dict(zip(X_train.columns, f_imp))
x_vals = list(feature_importance.keys())
y_vals = list(feature_importance.values())

fig = px.bar(
        x=x_vals,
        y=y_vals,
        title="Random Forest - Feature Importance",
        height=500,
        width=1000,
        labels={'y':'Importance Value', 'x': 'Feature'})
fig.update_xaxes(categoryorder='total descending')
fig.show()

In [84]:
# ROC Curve
fpr, tpr, thresholds = roc_curve(y_test_num, y_pred_rf)
roc_auc = auc(fpr, tpr)
fig = px.area(
    x=fpr, y=tpr,
    title=f'Random Forest - ROC Curve (AUC={roc_auc:.4f})',
    labels=dict(x='False Positive Rate', y='True Positive Rate'),
    width=700, height=500
)
fig.add_shape(
    type='line', line=dict(dash='dash'),
    x0=0, x1=1, y0=0, y1=1
)

fig.update_yaxes(scaleanchor="x", scaleratio=1)
fig.update_xaxes(constrain='domain')
fig.show()

 ### Modeling and Evaluation 4
 Analyze the results using your chosen method of evaluation. Use visualizations of the results to bolster the analysis. Explain any visuals and analyze why they are interesting to someone that might use this model.

### Modeling and Evaluation 5
Discuss the advantages of each model for each classification task, if any. If there are not advantages, explain why. Is any model better than another? Is the difference significant with 95% confidence? Use proper statistical comparison methods. You must use statistical comparison techniques—be sure they are appropriate for your chosen method of validation as discussed in unit 7 of the course.
_____________

__MORE DETAIL IS NEEDED ON WHY THIS IS BEST FOR OUR DATASET__

### Modeling and Evaluation 6
Which attributes from your analysis are most important? Use proper methods discussed in class to evaluate the importance of different attributes. Discuss the results and hypothesize about why certain attributes are more important than others for a given classification task.

### Deployment
How useful is your model for interested parties (i.e., the companies or organizations that might want to use it for prediction)? How would you measure the model's value if it was used by these parties? How would your deploy your model for interested parties? What other data should be collected? How often would the model need to be updated, etc.?
_________
__MORE DETAIL IS NEEDED ON WHY THIS IS BEST FOR OUR DATASET__

### Exceptional Work
You have free reign to provide additional analyses. One idea: grid search parameters in a parallelized fashion and visualize the performances across attributes. Which parameters are most significant for making a good model for each classification algorithm?

__Naive Bayes__